In [50]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import joblib 

In [51]:

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

In [52]:

def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    
    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0]
        landmark_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks.landmark])
        return landmark_array.flatten()  # Flatten to a 1D array
    return None

In [53]:
# Load images from directory and extract labels from folder names
def load_data(base_dir):
    data = []
    labels = []
    
    for emotion in os.listdir(base_dir):
        emotion_dir = os.path.join(base_dir, emotion)
        
        if os.path.isdir(emotion_dir):
            for img_file in os.listdir(emotion_dir):
                img_path = os.path.join(emotion_dir, img_file)
                landmarks = extract_landmarks(img_path)
                
                if landmarks is not None:
                    data.append(landmarks)
                    labels.append(emotion)  # Use folder name as label

    return np.array(data), np.array(labels)

In [54]:

# Load training and testing data
X_train, y_train = load_data('D:/Music_Recommender/data/3_emotion/train')
X_test, y_test = load_data('D:/Music_Recommender/data/3_emotion/test')


d:\Music_Recommender\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [55]:
# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [56]:
# Save the LabelEncoder for future use
joblib.dump(le, 'D:/Music_Recommender/Label_Encoder/label_encoder_3_emotion.pkl')

['D:/Music_Recommender/Label_Encoder/label_encoder_3_emotion.pkl']

Model 1

In [57]:
# Build the model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(y_train_encoded)), activation='softmax')  # Number of classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


d:\Music_Recommender\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
# Initialize early stopping
early_stopping = EarlyStopping(
    monitor='accuracy',  # Monitor validation accuracy
    patience=5,              # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value
)


In [59]:

# Train the model
model.fit(X_train, y_train_encoded, 
            epochs=50, 
            batch_size=32, 
            validation_split=0.2,  # Use 20% of training data for validation
            callbacks=[early_stopping])  # Add the early stopping callback


Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4703 - loss: 1.0231 - val_accuracy: 0.0000e+00 - val_loss: 2.1480
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5699 - loss: 0.8927 - val_accuracy: 0.0000e+00 - val_loss: 1.6694
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6959 - loss: 0.7919 - val_accuracy: 0.0000e+00 - val_loss: 1.2877
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7323 - loss: 0.6780 - val_accuracy: 0.0295 - val_loss: 1.0739
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7343 - loss: 0.6262 - val_accuracy: 0.2788 - val_loss: 0.9543
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7718 - loss: 0.5605 - val_accuracy: 0.5678 - val_loss: 0.8291
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7904 - loss: 0.5331 - val_accuracy: 0.2375 - val_loss: 1.1611
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7902 - loss: 0.5383 - val_accuracy: 0.3540

In [60]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7965 - loss: 0.4644  
Test Accuracy: 72.97%


In [61]:
# Save the model
model.save('D:/Music_Recommender/models/mediapipe_3emotion_model_1.h5')

Model 2


In [65]:
model_2 = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),  # Dropout layer for regularization
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(y_train_encoded)), activation='softmax')
])

# Compile the model
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [66]:
# Initialize early stopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=5,              # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value
)

# Train the model
model_2.fit(X_train, y_train_encoded, 
            epochs=50, 
            batch_size=32, 
            validation_split=0.2,  # Use 20% of training data for validation
            callbacks=[early_stopping])  # Add th

Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4429 - loss: 1.0478 - val_accuracy: 0.0000e+00 - val_loss: 1.5196
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4732 - loss: 0.9906 - val_accuracy: 0.0000e+00 - val_loss: 1.8705
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5207 - loss: 0.9445 - val_accuracy: 0.0000e+00 - val_loss: 1.8380
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5547 - loss: 0.8920 - val_accuracy: 0.0000e+00 - val_loss: 1.3448
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6339 - loss: 0.7825 - val_accuracy: 0.0000e+00 - val_loss: 1.4173
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6880 - loss: 0.7232 - val_accuracy: 0.0000e+00 - val_loss: 1.6369


In [64]:
# Evaluate the model
loss, accuracy = model_2.evaluate(X_test, y_test_encoded)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6212 - loss: 0.9348  
Test Accuracy: 35.52%


Model 3

In [68]:
# Model 3
# Build the model
model_3 = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(y_train_encoded)), activation='softmax')
])
# Compile the model
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [69]:
# Initialize early stopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=5,
    restore_best_weights=True
)
# Train the model
model_3.fit(X_train, y_train_encoded, 
            epochs=50, 
            batch_size=32, 
            validation_split=0.2,  # Use 20% of training data for validation
            callbacks=[early_stopping])  # Add the early stopping callback


Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6334 - loss: 0.8795 - val_accuracy: 0.0000e+00 - val_loss: 4.1707
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8015 - loss: 0.4991 - val_accuracy: 0.0310 - val_loss: 1.6851
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8113 - loss: 0.4879 - val_accuracy: 0.0000e+00 - val_loss: 4.9786
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8489 - loss: 0.3963 - val_accuracy: 0.0000e+00 - val_loss: 6.3825
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8551 - loss: 0.3804 - val_accuracy: 0.0059 - val_loss: 3.0135
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8474 - loss: 0.4109 - val_accuracy: 0.1962 - val_loss: 2.5501
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8415 - loss: 0.4012 - val_accuracy: 0.1121 - val_loss: 3.3324
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8383 - loss: 0.3955 - val_accuracy: 0.0000

In [70]:
# Evaluate the model
loss, accuracy = model_3.evaluate(X_test, y_test_encoded)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6884 - loss: 1.1898  
Test Accuracy: 47.49%


: 